# Data Validation with Pydantic: Base Model Extension ("ForwardRef" resolution fixed)

## Lamin Example

In [ ]:
import lamindb.schema as lns

In [ ]:
??lns.dev.sqlmodel.SQLModelModule

In [ ]:
??lns.dev.sqlmodel.SQLModelPrefix

In [ ]:
# Data validation works as expected (error raised for missing field)
pipeline = lns.Pipeline()

In [ ]:
# Data validation works as expected (no errors raised)

from datetime import datetime

pipeline = lns.Pipeline(created_at=datetime.now())
pipeline

In [ ]:
# Data validation now works for complex types!
run = lns.Run(name="Test Run", pipeline="invalid pipeline", created_at=datetime.now())
run.pipeline